In [24]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astroquery.gaia import Gaia
from astroquery.vizier import Vizier

In [2]:
# FGKM Data
job = Gaia.launch_job_async("SELECT source_id, lum_flame, teff_gspphot, mh_gspphot, age_flame, ebpminrp_gspphot \
FROM gaiadr3.gold_sample_fgkm_stars")

INFO: Query finished. [astroquery.utils.tap.core]


In [8]:
# Crossmatching source id's of FGKM sample with Gaia for radial velocity, radial velocity error (which is specified in Gaia's
# data release as the uncertainty value), and proper motion.
job2 = Gaia.launch_job_async("SELECT source_id, radial_velocity, radial_velocity_error, pm, phot_g_mean_mag \
FROM gaiadr3.gaia_source AS dr3 \
JOIN gaiadr3.gold_sample_fgkm_stars AS xmatch USING (source_id)")

INFO: Query finished. [astroquery.utils.tap.core]


In [9]:
# Checking results.
fgkm_new = job2.get_results()
print(fgkm_new)

     source_id      radial_velocity ...     pm    phot_g_mean_mag
                         km / s     ...  mas / yr       mag      
------------------- --------------- ... --------- ---------------
      8044474553216              -- ... 14.712925       14.947446
     12545600306304      -48.036938 ... 48.337337       12.194942
     14401026146688              -- ... 6.6363807       14.904602
     15637976759168      -34.489674 ... 1.9501954       13.135885
     16531329957376      -25.610313 ...  89.93515      10.0337925
     16733192740608              -- ... 23.467905        9.925234
     26834955821312       24.510994 ... 27.242292      10.0179405
     30034707100800      -32.520874 ...  15.30033       14.038215
     38349763812096              -- ... 7.8729506       15.268802
     38693361197568      -20.947187 ...  6.599868       13.818121
                ...             ... ...       ...             ...
6347928441918018432       46.322063 ... 20.121223       13.352476
6347929197

In [4]:
# Gaia's archives do not have a specific data column or source for proper motion uncertainty that I can crossmatch with 
# (they do have some for error though). However, on the webpage detailing the Gaia Data Release 3 contents and limitations, 
# they do specifically mention that the median proper motion uncertainties are 0.02-0.03 mas/yr for G<15, 0.07 mas/yr at 
# G=17, 0.5 mas/yr at G=20, and 1.4 mas/yr at G=21 mag. These G magnitudes were crossmatched through phot_g_mean_mag in 
# the Gaia archive, in case this information is useful.

In [29]:
# Checking for the Bailer-Jones distance catalog's existence within Gaia.
distance_table = Gaia.load_table("external.gaiaedr3_distance")
print(distance_table)

Retrieving table 'external.gaiaedr3_distance'
TAP Table name: external.external.gaiaedr3_distance
Description: 
Bailer-Jones et al. (2021) EDR3 distances. <p> Estimating distances from parallaxes. V: Geometric and photogeometric distances to 1.47 billion stars in Gaia Early Data Release 3.
Bailer-Jones et al. 2021 AJ 161 147. Data replicated from gedr3dist.main table at GAVO Data Center TAP service https://dc.g-vo.org/tap and TAP metadata as of December 2020.
<p>
    Original table description.
<p>
    We estimate the distance from the Sun to sources in Gaia EDR3 that have
    parallaxes. We provide two types of distance estimate, together with
    their corresponding asymmetric uncertainties, using Bayesian posterior
    density functions that we sample for each source. Our prior is based
    on a detailed model of the 3D spatial, colour, and magnitude
    distribution of stars in our Galaxy that includes a 3D map of
    interstellar extinction.
<p>
    The first type of distance esti

In [31]:
# Looking at the column metadata for the catalog's table.
for column in distance_table.columns:
    print(column.name)

source_id
r_med_geo
r_lo_geo
r_hi_geo
r_med_photogeo
r_lo_photogeo
r_hi_photogeo
flag


In [32]:
# Crossmatching with the FGKM sample.
job4 = Gaia.launch_job_async("SELECT source_id, r_med_geo, r_lo_geo, r_hi_geo, r_med_photogeo, r_lo_photogeo, r_hi_photogeo \
FROM external.gaiaedr3_distance \
JOIN gaiadr3.gold_sample_fgkm_stars AS xmatch USING (source_id)")

INFO: Query finished. [astroquery.utils.tap.core]


In [33]:
# Checking results.
fgkm_distance = job4.get_results()
print(fgkm_distance)

     source_id      r_med_geo  r_lo_geo ... r_lo_photogeo r_hi_photogeo
                        pc        pc    ...       pc            pc     
------------------- --------- --------- ... ------------- -------------
      8044474553216 590.81683  580.0101 ...     583.34924      603.8781
     12545600306304  230.1045 228.83165 ...     229.11658     231.29326
     14401026146688  785.2145  772.1763 ...       768.195      802.9816
     15637976759168  732.0263  724.8018 ...      722.2218     738.58264
     16531329957376 116.74163 116.45008 ...     116.45989     116.90652
     16733192740608 181.79086 181.07738 ...     181.02267     182.45615
     26834955821312  167.1039 166.75528 ...     166.73569     167.44814
     30034707100800  942.9716  928.0086 ...     926.12695      957.4273
     38349763812096 463.56647 456.65765 ...     456.35257      469.8667
     38693361197568 863.81824  853.5123 ...      850.7704     880.17944
                ...       ...       ... ...           ...       

In [ ]:
# Previous, failed attempts at crossmatching for the Bailer-Jones catalog. This can be ignored, but I've kept it here in case
# it has some refernce use for myself later.

#job3 = Gaia.launch_job_async("SELECT source_id, ra, dec, r_med_geo, r_lo_geo, r_hi_geo, r_med_photogeo, r_lo_photogeo, \
#r_hi_photogeo, phot_bp_mean_mag-phot_rp_mean_mag AS bp_rp, phot_g_mean_mag-5*LOG10(r_med_geo)+5 AS qg_geo, \
#phot_g_mean_mag-5*LOG10(r_med_photogeo)+5 AS gq_photogeo \
#FROM gedr3dist.main \
#JOIN gaiadr3.gold_sample_fgkm_stars AS xmatch USING (source_id)")